# Tutorial Notebook

Ok this is supposed to be a tutorial python notebook to import and use functions from be-scan but I can't figure out the syntax of how to import, and it's not a priority

In [2]:
# from be-scan import sgrna.findall_be